In [271]:
# Import necessary libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime


def get_match_data(start_date, time_now_formatted, devengine):
    # Get historical match data on hard surface between start date and yesterday
    elo_hard = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Hard where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )

    # Get historical match data on clay surface between start date and yesterday
    elo_clay = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Clay where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )
    # Get historical match data on clay surface between start date and yesterday
    elo_grass = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_grass where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )
    # Get today's matches on hard surface that haven't yet been resulted
    elo_data_hard = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Hard where Date like '{time_now_formatted}' --and resulted like 'False'",
        con=devengine,
    )

    # Get today's matches on clay surface that haven't yet been resulted
    elo_data_clay = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Clay where Date like '{time_now_formatted}' --and resulted like 'False'",
        con=devengine,
    )

    # Get today's matches on clay surface that haven't yet been resulted
    elo_data_grass = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_grass where Date like '{time_now_formatted}' --and resulted like 'False'",
        con=devengine,
    )

    # Get historical match data on clay surface between start date and yesterday
    elo_grass = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_grass where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )    
    
    # Get historical match data on clay surface between start date and yesterday
    elo_all = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_All where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )

    # Get today's matches on clay surface that haven't yet been resulted
    elo_data_all = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_All where Date like '{time_now_formatted}' --and resulted like 'False'",
        con=devengine,
    )

    return elo_hard, elo_clay, elo_data_hard, elo_data_clay, elo_grass, elo_data_grass, elo_all, elo_data_all


# Connect to SQLite database using SQLAlchemy's create_engine
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
# Get current date and time
time_now = datetime.datetime.now()

# Format current date as string in YYYY-MM-DD format
time_now_formatted = time_now.strftime("%Y-%m-%d")

# Get the start date two years ago from today
today = time_now
two_years_ago = (today - datetime.timedelta(days=365 * 2)).strftime("%Y-%m-%d")

(
    elo_hard,
    elo_clay,
    elo_data_hard,
    elo_data_clay,
    elo_grass,
    elo_data_grass,
    elo_all,
    elo_data_all,    
) = get_match_data(two_years_ago, time_now_formatted, devengine)


def get_player_record(player, opponent_rank, history, range_low, range_high, auto):
    if auto:
        opponent_rank_low = opponent_rank - range_low
        opponent_rank_high = opponent_rank + range_high
    else:
        opponent_rank_low = range_low
        opponent_rank_high = range_high

    player_history = history[
        (
            (history["Fav"] == player)
            & (
                (history["Dog_Rank"] > opponent_rank_low)
                & (history["Dog_Rank"] < opponent_rank_high)
            )
        )
        | (
            (history["Dog"] == player)
            & (
                (history["Fav_Rank"] > opponent_rank_low)
                & (history["Fav_Rank"] < opponent_rank_high)
            )
        )
    ]
    if player_history.empty == False:
        result = float(
            len(player_history[player_history["Winner"] == player])
            / len(player_history)
        )
        return result, len(player_history)
    else:
        return 0, 0


def get_filtered_data(elo_data, elo):
    result_df = pd.DataFrame()
    for _, row in elo_data.sort_values(by="Time").iterrows():
        low_limit = 50
        high_limit = 50

        fav_percent, games = get_player_record(
            row.Fav, row.Dog_Rank, elo, low_limit, high_limit, True
        )
        count = 0
        while games < 10 and count < 200:
            count = count + 1
            low_limit = low_limit + 10
            high_limit = high_limit + 10
            fav_percent, games = get_player_record(
                row.Fav, row.Dog_Rank, elo, low_limit, high_limit, True
            )

        low_limit = 50
        high_limit = 50
        dog_percent, games2 = get_player_record(
            row.Dog, row.Fav_Rank, elo, low_limit, high_limit, True
        )
        count = 0
        while games2 < 10 and count < 200:
            count = count + 1
            low_limit = low_limit + 10
            high_limit = high_limit + 10
            dog_percent, games2 = get_player_record(
                row.Dog, row.Fav_Rank, elo, low_limit, high_limit, True
            )

        # New code to calculate player's record against rank 0 to 100
        fav_record, _ = get_player_record(
            row.Fav, 100, elo, 0, 100, False
        )
        dog_record, _ = get_player_record(
            row.Dog, 100, elo, 0, 100, False
        )
        
        if games > 4 and games2 > 4:
            temp_df = pd.DataFrame(
                {
                    "Time": [row.Time],
                    "Fav_Odds": [row.Fav_Odds],
                    "Dog_Odds": [row.Dog_Odds],
                    "Fav": [row.Fav],
                    "Elo_Fav": [row.Elo_Fav],
                    "Fav_Record": ["{:.0%}".format(fav_percent)],
                    "Fav_Games": [games],
                    "Dog": [row.Dog],
                    "Dog_Odds": [row.Dog_Odds],
                    "Dog_Record": ["{:.0%}".format(dog_percent)],
                    "Dog_Games": [games2],
                    "fav_percent": [fav_percent],
                    "dog_percent": [dog_percent],
                    "Sex": [row.Sex],
                    "Resulted": [row.Resulted],
                    "fav_rank": [row.Fav_Rank],
                    "dog_rank": [row.Dog_Rank],
                    "Elo_Fav_Elo": [row.Elo_Fav_Elo],
                    "Elo_Dog_Elo": [row.Elo_Dog_Elo],
                    "Fav_Top100": [round(fav_record,1)],  # New column
                    "Dog_Top100": [round(dog_record,1)]  # New column
                }
            )
            result_df = pd.concat([result_df, temp_df])
    return result_df


results_hard = get_filtered_data(elo_data_hard, elo_hard)
results_clay = get_filtered_data(elo_data_clay, elo_clay)
results_grass = get_filtered_data(elo_data_grass, elo_grass)
#results_grass = get_filtered_data(elo_data_grass, elo_hard)
results_all = get_filtered_data(elo_data_all, elo_all)


def process_serving_data(result_df):
    try:
        # Try to read the 'servers_today.xlsx' file
        serving = pd.read_csv("mensserving.csv")

        # Try to read the 'servers_today_womens.xlsx' file
        serving_womens = pd.read_csv("womensserving.csv")

        # If 'serving_womens' dataframe is not empty, concatenate with 'serving' dataframe
        if serving_womens.empty == False:
            serving = pd.concat([serving, serving_womens])
        else:
            serving = serving

        # Drop the 'Time' column from the 'serving' dataframe
        # serving = serving.drop(columns='Time')
    except FileNotFoundError as e:
        # If either of the excel files is not found, print an error message and set serving to None
        print("The required excel file could not be found.")
        print("Error:", e)
        serving = None

    if serving is not None:
        try:
            # Try to merge the 'result_df' and 'serving' dataframes on the 'Fav' and 'Dog' columns
            result = pd.merge(
                result_df, serving, how="left", left_on=["Fav"], right_on=["Name"]
            )
            result = pd.merge(
                result, serving, how="left", left_on=["Dog"], right_on=["Name"]
            )
            result.rename(
                columns={
                    "Service Games Won_x": "Fav_Serve%",
                    "Service Games Won_y": "Dog_Serve%",
                    "Return Games Won_x": "Fav_Return%",
                    "Return Games Won_y": "Dog_Return%",
                },
                inplace=True,
            )
            result.drop(columns=["Name_x", "Name_y"], inplace=True)

            # Set 'final_hard' to the 'result' dataframe
            result_serving = result
        except Exception as e:
            # If an error occurs during merging, print an error message and set both 'result' and 'final_hard' to None
            print("Error occured while merging the dataframes.")
            print("Error:", e)
            result = None
            result_serving = None
    else:
        result = None
        result_serving = None

    return result, result_serving


_, serving_hard = process_serving_data(results_hard)
_, serving_clay = process_serving_data(results_clay)
_, serving_grass = process_serving_data(results_grass)
_, serving_all = process_serving_data(results_all)


data_concat = pd.DataFrame(columns=["Date", "Player", "Odds", "Win/Loss"])
for dataset_type in [("Winner", "Win"), ("Loser", "Loss")]:
    df = elo_hard[["Date", dataset_type[0], f"{dataset_type[0]}_Odds"]].copy()
    df["Player"] = df[dataset_type[0]]
    df["Odds"] = df[f"{dataset_type[0]}_Odds"]
    df["Win/Loss"] = dataset_type[1]
    df.drop(columns=[dataset_type[0], f"{dataset_type[0]}_Odds"], inplace=True)
    data_concat = pd.concat([data_concat, df])
data_concat = data_concat.sort_index()
data_concat["Odds"] = data_concat.Odds.astype(float)


def analyse_matchups(result_df, data_concat):
    for _, matchup in result_df.iterrows():
        player1 = matchup.Fav
        player2 = matchup.Dog
        player1_odds = float(matchup.Fav_Odds)
        player1_odds_hi = player1_odds + 0.15
        player1_odds_lo = player1_odds - 0.15
        player2_odds = float(matchup.Dog_Odds)
        player2_odds_hi = player2_odds + 0.15
        player2_odds_lo = player2_odds - 0.15
        player1 = data_concat[data_concat["Player"] == player1].copy()
        player2 = data_concat[data_concat["Player"] == player2].copy()
        player2 = player2[
            (player2["Odds"] > player2_odds_lo) & (player2["Odds"] < player2_odds_hi)
        ]
        if len(player2) > 0:
            winperc2 = len(player2[player2["Win/Loss"] == "Win"]) / len(player2)
        else:
            winperc2 = 0
        player1 = player1[
            (player1["Odds"] > player1_odds_lo) & (player1["Odds"] < player1_odds_hi)
        ]
        if len(player1) > 0:
            winperc1 = len(player1[player1["Win/Loss"] == "Win"]) / len(player1)
        else:
            winperc1 = 0
        if len(player1) > 5 and len(player2) > 3:
            print(
                matchup.Time,
                f"{matchup.Fav} ({round(player1_odds_lo,2)}-->{round(player1_odds_hi,2)})",
                f"{matchup.Dog} ({round(player2_odds_lo,2)}-->{round(player2_odds_hi,2)})",
            )
            print(len(player1), winperc1, len(player2), winperc2)


# analyse_matchups(serving_clay,data_concat)
final_hard = serving_hard
final_clay = serving_clay
final_grass = serving_grass
final_all = serving_all

Error occured while merging the dataframes.
Error: 'Fav'


In [272]:
def last_five(df,pastmatches):
    for index, row in df.iterrows():
        fav=row.Fav
        dog=row.Dog
        last_five_matches_fav=pastmatches[(pastmatches['Winner']==fav)|(pastmatches['Loser']==fav)].tail(5)
        if len(last_five_matches_fav)>0:  
            fav_last_five_win_perc=len(last_five_matches_fav[last_five_matches_fav['Winner']==fav])/len(last_five_matches_fav)
        else:
            fav_last_five_win_perc=0
        last_five_matches_dog=pastmatches[(pastmatches['Winner']==dog)|(pastmatches['Loser']==dog)].tail(5)
        if len(last_five_matches_dog)>0:    
            dog_last_five_win_perc=len(last_five_matches_dog[last_five_matches_dog['Winner']==dog])/len(last_five_matches_dog) 
        else:
            dog_last_five_win_perc=0
        df.at[index, 'fav_last_five_win_perc'] = fav_last_five_win_perc
        df.at[index, 'dog_last_five_win_perc'] = dog_last_five_win_perc
    return df

if final_hard is not None:
    final_hard=last_five(final_hard,elo_hard)
    final_hard['Fav_Odds']=final_hard['Fav_Odds'].astype(float)
    final_hard['Dog_Odds']=final_hard['Dog_Odds'].astype(float)    

In [273]:
if final_clay is not None:
    final_clay=last_five(final_clay,elo_clay)
    final_clay['Dog_Odds']=final_clay['Dog_Odds'].astype(float)
    final_clay['Fav_Odds']=final_clay['Fav_Odds'].astype(float)

In [274]:
if final_grass is not None:
    final_grass=last_five(final_grass,elo_grass)
    final_grass['Fav_Odds']=final_grass['Fav_Odds'].astype(float)
    final_grass['Dog_Odds']=final_grass['Dog_Odds'].astype(float)  
final_all=last_five(final_all,elo_all)
final_all['Fav_Odds']=final_all['Fav_Odds'].astype(float)
final_all['Dog_Odds']=final_all['Dog_Odds'].astype(float)        

In [275]:
for _,i in elo_data_hard.iterrows():
    check1=elo_hard[((elo_hard['Winner']==i.Winner)&(elo_hard['Loser']==i.Loser))|((elo_hard['Loser']==i.Winner)&(elo_hard['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

Jessica Pegula beat Yulia Putintseva
Iga Swiatek beat Karolina Pliskova
Liudmila Samsonova beat Qinwen Zheng
Liudmila Samsonova beat Qinwen Zheng
Liudmila Samsonova beat Qinwen Zheng
Qinwen Zheng beat Liudmila Samsonova
Karolina Muchova beat Sorana Cirstea
Sorana Cirstea beat Karolina Muchova
Maria Sakkari beat Danielle Collins
Beatriz Haddad Maia beat Leylah Fernandez
Andrey Rublev beat Mackenzie Mcdonald
Daniil Medvedev beat Matteo Arnaldi


In [276]:
for _,i in elo_data_clay.iterrows():
    check1=elo_clay[((elo_clay['Winner']==i.Winner)&(elo_clay['Loser']==i.Loser))|((elo_clay['Loser']==i.Winner)&(elo_clay['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

Sumit Nagal beat Benjamin Hassan
Francesco Maestrelli beat Riccardo Bonadio
Dimitar Kuzmanov beat Michael Geerts
Kimmer Coppejans beat Nino Serdarusic


In [277]:
#for _,i in elo_data_grass.iterrows():
#    check1=elo_grass[((elo_grass['Winner']==i.Winner)&(elo_grass['Loser']==i.Loser))|((elo_grass['Loser']==i.Winner)&(elo_grass['Winner']==i.Loser))]
#    if check1.empty==False:
#        for _, x in check1.iterrows():
#            print(f"{x.Winner} beat {x.Loser}")

In [278]:
for _,i in elo_data_all.iterrows():
    check1=elo_all[((elo_grass['Winner']==i.Winner)&(elo_all['Loser']==i.Loser))|((elo_all['Loser']==i.Winner)&(elo_all['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

Francesco Maestrelli beat Riccardo Bonadio
Maria Sakkari beat Danielle Collins
Qinwen Zheng beat Liudmila Samsonova
Karolina Muchova beat Sorana Cirstea
Andy Murray beat Max Purcell
Andrey Rublev beat Mackenzie Mcdonald
Daniil Medvedev beat Matteo Arnaldi
Francisco Cerundolo beat Tommy Paul
Francisco Cerundolo beat Tommy Paul
Tereza Martincova beat Dayana Yastremska


In [279]:
if final_hard is not None:
    final_hard=final_hard

else:
    final_hard=pd.DataFrame(columns=['Fav_Top100','Dog_Top100','Sex','Resulted','Time','Fav','fav_rank','dog_rank','Fav_Odds','fav_percent','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','Dog_Serve%','Dog_Return%','dog_last_five_win_perc'])

final_hard[(final_hard['Resulted']=='False')][['Time','Fav_Top100','Dog_Top100','Fav','fav_rank','dog_rank','Fav_Odds','fav_percent','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav_Top100,Dog_Top100,Fav,fav_rank,dog_rank,Fav_Odds,fav_percent,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,Dog_Serve%,Dog_Return%,dog_last_five_win_perc
1,01:10,0.6,0.7,Belinda Bencic,13.0,47.0,1.13,0.631579,0.77,0.32,0.6,Alycia Parks,5.97,0.545455,0.73,0.24,0.4
3,01:15,0.8,0.5,Taylor Fritz,9.0,34.0,1.33,0.738095,0.84,0.24,0.8,Ugo Humbert,3.38,0.428571,0.76,0.17,0.6
4,02:35,0.7,0.4,Jessica Pegula,3.0,61.0,1.13,0.837838,0.73,0.37,0.6,Yulia Putintseva,6.11,0.357143,0.65,0.37,0.2
5,02:40,0.9,0.6,Iga Swiatek,1.0,23.0,1.08,0.865385,0.76,0.50,1.0,Karolina Pliskova,8.41,0.518519,0.71,0.31,0.4
6,02:40,0.7,0.6,Liudmila Samsonova,18.0,24.0,1.69,0.684211,0.81,0.31,0.6,Qinwen Zheng,2.17,0.551724,0.75,0.26,0.4
7,03:00,0.7,0.6,Hubert Hurkacz,17.0,44.0,1.33,0.666667,0.91,0.18,0.4,Miomir Kecmanovic,3.35,0.560000,0.79,0.25,0.4
8,03:15,0.5,0.4,Lorenzo Musetti,19.0,86.0,1.69,0.611111,0.78,0.24,0.2,Thanasi Kokkinakis,2.18,0.350000,0.84,0.14,0.8
9,03:55,0.8,0.2,Daniil Medvedev,3.0,66.0,1.11,0.875000,0.86,0.27,1.0,Matteo Arnaldi,6.93,0.545455,0.80,0.13,0.6
10,04:00,0.5,0.6,Daria Kasatkina,14.0,34.0,1.29,0.514286,0.61,0.47,0.4,Anna Blinkova,3.60,0.454545,0.63,0.33,0.4
11,04:00,0.6,0.5,Karolina Muchova,17.0,31.0,1.47,0.619048,0.71,0.25,0.8,Sorana Cirstea,2.70,0.480000,0.65,0.38,0.6


In [280]:
if final_clay is not None:
    final_clay=final_clay

else:
    final_clay=pd.DataFrame(columns=['Fav_Top100','Dog_Top100','Sex','Resulted','Time','Fav','fav_rank','dog_rank','Fav_Odds','fav_percent','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','Dog_Serve%','Dog_Return%','dog_last_five_win_perc'])

final_clay[(final_clay['Sex']=='Womens')&(final_clay['Resulted']=='False')][['Time','Fav_Top100','Dog_Top100','Fav','fav_rank','dog_rank','Fav_Odds','fav_percent','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav_Top100,Dog_Top100,Fav,fav_rank,dog_rank,Fav_Odds,fav_percent,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,Dog_Serve%,Dog_Return%,dog_last_five_win_perc


In [281]:
final_clay[(final_clay['Sex']=='Mens')&(final_clay['Resulted']=='False')][['Time','Fav_Top100','Dog_Top100','Fav','fav_rank','dog_rank','Fav_Odds','fav_percent','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav_Top100,Dog_Top100,Fav,fav_rank,dog_rank,Fav_Odds,fav_percent,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,Dog_Serve%,Dog_Return%,dog_last_five_win_perc
0,02:10,0.5,0.0,Thiago Agustin Tirante,134.0,342.0,1.40,0.818182,0.00,0.00,0.2,Gustavo Heide,2.84,0.300000,0.00,0.00,0.4
3,02:40,0.3,0.0,Mariano Navone,177.0,334.0,1.42,0.785714,0.00,0.00,0.8,Javier Barranco Cosano,2.74,0.400000,0.00,0.00,0.4
4,03:40,0.2,0.7,Riccardo Bonadio,199.0,249.0,1.73,0.535714,0.00,0.00,0.6,Francesco Maestrelli,2.04,0.411765,0.67,0.08,0.2
5,04:00,0.2,0.0,Felipe Meligeni Alves,168.0,329.0,1.17,0.769231,0.14,0.25,0.8,Eduardo Ribeiro,4.83,0.400000,0.00,0.00,0.2
6,05:00,0.2,0.0,Alberto Olivieri Genaro,193.0,530.0,1.17,0.800000,NaN,NaN,0.2,Ignacio Monzon,4.72,0.181818,0.00,0.00,0.2
7,05:00,0.4,1.0,Federico Coria,103.0,279.0,1.40,0.684211,0.68,0.25,0.4,Juan Pablo Ficovich,2.83,0.600000,0.78,0.16,0.4
8,05:30,0.0,0.0,Lucas Reis Da Silva Joao,288.0,326.0,1.57,0.363636,NaN,NaN,0.2,Bernard Tomic,2.31,0.285714,0.00,0.00,0.2
9,19:00,0.0,0.5,Sumit Nagal,183.0,253.0,1.39,0.571429,0.00,0.00,0.6,Benjamin Hassan,2.87,0.461538,0.00,0.00,0.4
10,20:00,0.0,0.0,Hernan Casanova,247.0,572.0,1.52,0.700000,0.67,0.00,0.6,Federico Iannaccone,2.37,0.300000,NaN,NaN,0.6
11,20:00,0.4,0.0,Kimmer Coppejans,160.0,308.0,1.53,0.666667,0.00,0.00,0.4,Nino Serdarusic,2.39,0.500000,0.00,0.00,0.2


In [282]:
if final_hard is not None:
    final_hard.to_pickle('Hard_Today')
else:
    final_clay[final_clay['Sex']=='k'].to_pickle('Hard_Today')
if final_clay is not None:
    final_clay.to_pickle('Clay_Today')  
final_all.to_pickle('All_Today')    
final_clay=final_clay[(final_clay['Resulted']=='False')].copy()
final_hard=final_hard[(final_hard['Resulted']=='False')].copy()    

In [283]:
final_all[(final_all['Sex']=='Mens')&(final_all['Resulted']=='False')][['Time','Fav_Top100','Dog_Top100','Fav','fav_rank','dog_rank','Fav_Odds','fav_percent','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav_Top100,Dog_Top100,Fav,fav_rank,dog_rank,Fav_Odds,fav_percent,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,Dog_Serve%,Dog_Return%,dog_last_five_win_perc
4,01:15,0.7,0.4,Taylor Fritz,9.0,34.0,1.33,0.705882,0.84,0.24,0.8,Ugo Humbert,3.38,0.333333,0.76,0.17,0.6
6,02:10,0.4,0.0,Thiago Agustin Tirante,134.0,342.0,1.40,0.600000,0.00,0.00,0.2,Gustavo Heide,2.84,0.300000,0.00,0.00,0.4
12,02:40,0.3,0.0,Mariano Navone,177.0,334.0,1.42,0.785714,0.00,0.00,0.8,Javier Barranco Cosano,2.74,0.400000,0.00,0.00,0.4
13,03:00,0.6,0.5,Hubert Hurkacz,17.0,44.0,1.33,0.632353,0.91,0.18,0.6,Miomir Kecmanovic,3.35,0.477273,0.79,0.25,0.6
14,03:15,0.6,0.4,Lorenzo Musetti,19.0,86.0,1.69,0.658537,0.78,0.24,0.6,Thanasi Kokkinakis,2.18,0.344828,0.84,0.14,0.8
15,03:40,0.2,0.4,Riccardo Bonadio,199.0,249.0,1.73,0.580645,0.00,0.00,0.6,Francesco Maestrelli,2.04,0.551724,0.67,0.08,0.2
16,03:55,0.8,0.3,Daniil Medvedev,3.0,66.0,1.11,0.857143,0.86,0.27,0.8,Matteo Arnaldi,6.93,0.272727,0.80,0.13,0.8
17,04:00,0.1,0.0,Felipe Meligeni Alves,168.0,329.0,1.17,0.769231,0.14,0.25,0.4,Eduardo Ribeiro,4.83,0.400000,0.00,0.00,0.2
20,04:05,0.7,0.4,Stefanos Tsitsipas,4.0,276.0,1.27,0.700000,0.86,0.21,0.8,Gael Monfils,3.83,0.545455,0.84,0.28,0.6
21,04:05,0.4,0.5,Gijs Brouwer,138.0,272.0,1.52,0.562500,0.85,0.23,0.6,Ryan Peniston,2.46,0.260870,0.80,0.19,0.2


In [284]:
final_all[(final_all['Sex']=='Womens')&(final_all['Resulted']=='False')][['Time','Fav_Top100','Dog_Top100','Fav','fav_rank','dog_rank','Fav_Odds','fav_percent','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav_Top100,Dog_Top100,Fav,fav_rank,dog_rank,Fav_Odds,fav_percent,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,Dog_Serve%,Dog_Return%,dog_last_five_win_perc
2,01:10,0.7,0.5,Belinda Bencic,13.0,47.0,1.13,0.681818,0.77,0.32,0.6,Alycia Parks,5.97,0.380952,0.73,0.24,0.6
3,01:15,0.3,0.0,Maja Chwalinska,532.0,328.0,1.69,0.500000,0.67,0.36,0.2,Anna Siskova,2.15,0.714286,0.00,0.00,0.6
9,02:35,0.7,0.4,Jessica Pegula,3.0,61.0,1.13,0.857143,0.73,0.37,0.6,Yulia Putintseva,6.11,0.333333,0.65,0.37,0.4
10,02:40,0.9,0.6,Iga Swiatek,1.0,23.0,1.08,0.867470,0.76,0.50,1.0,Karolina Pliskova,8.41,0.500000,0.71,0.31,0.4
11,02:40,0.6,0.6,Liudmila Samsonova,18.0,24.0,1.69,0.555556,0.81,0.31,0.6,Qinwen Zheng,2.17,0.540000,0.75,0.26,0.8
18,04:00,0.6,0.5,Karolina Muchova,17.0,31.0,1.47,0.555556,0.71,0.25,0.6,Sorana Cirstea,2.70,0.428571,0.65,0.38,0.6
19,04:00,0.6,0.5,Daria Kasatkina,14.0,34.0,1.29,0.573529,0.61,0.47,0.6,Anna Blinkova,3.60,0.450000,0.63,0.33,0.6
30,05:30,0.4,0.6,Leylah Fernandez,81.0,12.0,1.87,0.321429,0.73,0.24,0.6,Beatriz Haddad Maia,1.94,0.680851,0.72,0.31,0.6
36,09:00,0.7,0.6,Aryna Sabalenka,2.0,35.0,1.16,0.657895,0.68,0.37,0.8,Petra Martic,5.39,0.482759,0.74,0.29,0.4
37,09:00,0.6,0.4,Maria Sakkari,8.0,48.0,1.58,0.589041,0.75,0.33,0.6,Danielle Collins,2.39,0.392857,NaN,NaN,0.6
